### This notebook is kindly provided by the data ambassador of the DSSG Berlin for a smooth start to the project and to give an overview of the data

# Basic Data Analysis of the DRK Jobs dataset

## Loading the dataset

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
df = pd.read_parquet("DRK_Jobs_basic_merged.parquet.gzip")

## Exploring the dataset

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.nunique()

In [ ]:
df.head()

In [ ]:
df.iloc[0]

In [ ]:
df["subject.createdOn"] = pd.to_datetime(df["subject.createdOn"])

## Basic Data Analysis

In [ ]:
df.shape

> There are **1807** job openings in the raw dataset

In [ ]:
employer_job_counts = df["subject.employer.meta.name.value"].value_counts().to_frame()

In [ ]:
employer_job_counts["cumsum"] = 100/df.shape[0] * employer_job_counts["subject.employer.meta.name.value"].cumsum()

In [ ]:
employer_job_counts.shape

> There are **234** unique employers posting jobs in the portal. More than 85% of these employers have less than 10 job openings.

In [ ]:
fig, ax = plt.subplots(figsize = (5,5), dpi = 100)
employer_job_counts["subject.employer.meta.name.value"].hist(bins = 10, weights = 100 * np.ones(employer_job_counts.shape[0])/employer_job_counts.shape[0], ax=ax)
ax.set_xlabel("Number of job listings per employer")
ax.set_ylabel("Percentage of total number of employers")
ax.set_title("Histogram of the number of job postings per employer");

In [ ]:
fig, ax = plt.subplots(figsize = (6,4), dpi = 100)
employer_job_counts["subject.employer.meta.name.value"].head(20).plot.barh(ax= ax)
ax.set_xlabel("Number of job listings")
ax.invert_yaxis()
ax.set_title("Top 20 employers by number of open job postings");

In [ ]:
city_job_counts = df["subject.location.city"].value_counts().to_frame()

In [ ]:
city_job_counts["cumsum"] = 100/df.shape[0] * city_job_counts["subject.location.city"].cumsum()

In [ ]:
city_job_counts.shape

> There are **360** unique cities with job openings in the portal. More than 90% of these employers have less than 10 job openings.

In [ ]:
fig, ax = plt.subplots(figsize = (5,5), dpi = 100)
city_job_counts["subject.location.city"].hist(bins = 10, weights = 100 * np.ones(city_job_counts.shape[0])/city_job_counts.shape[0], ax=ax)
ax.set_xlabel("Number of job listings per city")
ax.set_ylabel("Percentage of total number of cities")
ax.set_title("Histogram of the number of job postings per city");

In [ ]:
fig, ax = plt.subplots(figsize = (6,4), dpi = 100)
city_job_counts["subject.location.city"].head(20).plot.barh(ax= ax)
ax.set_xlabel("Number of job listings")
ax.invert_yaxis()
ax.set_title("Top 20 cities by number of open job postings");

In [ ]:
fig, ax = plt.subplots(figsize = (6,4), dpi = 100)
df["subject.employmentTypes"].value_counts().plot.barh(ax= ax)
ax.set_xlabel("Number of job listings")
ax.invert_yaxis()
ax.set_title("Employment Type distribution of job postings");

### Page Views for job openings

> Most of the job postings have less than **50 page views**. 

In [ ]:
fig, ax = plt.subplots(figsize = (5,5), dpi = 100)
df["Page_Views"].plot.hist(bins=20, ax=ax,weights = 100 * np.ones(df.shape[0])/df.shape[0])
ax.set_xlabel("Number of page views per job posting")
ax.set_ylabel("Percentage of job postings")
ax.set_title("Histogram of the number of page views per job posting");

In [ ]:
fig, ax = plt.subplots(figsize = (5,5), dpi = 100)
df.plot.scatter("Page_Views", "Unique_Page_Views", ax=ax)
ax.set_xlabel("Number of page views per job posting")
ax.set_ylabel("Number of unique page views per job posting")
ax.set_title("Distribution of number of page views vs unique views per job posting");

### Top 10 pages by the number of views

In [ ]:
df.sort_values("Page_Views", ascending=False).head(10)

### Number of applicants to job openings

> Close to **95%** of the applications do not have any applications yet. Around **210** jobs of the **1419** listed have had at least one applicant till now. 

In [ ]:
fig, ax = plt.subplots(figsize = (5,5), dpi = 100)
df["Goal_Completions"].plot.hist(bins=20, ax=ax,weights = 100 * np.ones(df.shape[0])/df.shape[0])
ax.set_xlabel("Number of applications per job posting")
ax.set_ylabel("Percentage of job postings")
ax.set_title("Histogram of the number of applications per job posting");

### Top 10 pages by the number of submitted applications

In [ ]:
df.sort_values("Goal_Completions", ascending=False).head(10)

In [ ]:
fig, ax = plt.subplots(figsize = (6,4), dpi = 100)
df.groupby("subject.location.city")["Goal_Completions"].sum().sort_values().tail(20).plot.barh(ax=ax)
ax.set_xlabel("Number of applications")
ax.set_ylabel("")
ax.set_title("Top 20 cities by number of job applications");

### Distribution of number of applications vs number of job postings. 

For 161 jobs there was only one applicant, for 27 jobs there were 2 applicants etc.

In [ ]:
df.query("`Goal_Completions` > 0")["Goal_Completions"].value_counts().to_frame().reset_index().rename(columns={'index': 'Number of applications', 'Goal_Completions': 'Number of job postings'})

In [ ]:
# import feather
# df.to_feather("DRK_Jobs_basic_merged.feather", pyarrow.feather.write_feather())

In [ ]:
df.to_csv("DRK.csv", index = False)